In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# ---------------------
# 1. Data Preparation
# ---------------------
# Simulating a recyclable items dataset (paper, glass, plastic, metal)
# In real deployment, use actual images from edge device cameras
NUM_CLASSES = 4
CLASS_NAMES = ['paper', 'glass', 'plastic', 'metal']

# Generate synthetic dataset (replace with real images in production)
def generate_synthetic_data(num_samples=1000):
    images = np.random.randint(0, 256, (num_samples, 96, 96, 3), dtype=np.uint8)
    labels = np.random.randint(0, NUM_CLASSES, num_samples)
    return images, labels

train_images, train_labels = generate_synthetic_data(800)
test_images, test_labels = generate_synthetic_data(200)

# Preprocessing pipeline
def preprocess(images, labels):
    images = tf.cast(images, tf.float32) / 255.0
    images = tf.image.resize(images, (96, 96))
    return images, labels

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(preprocess).shuffle(100).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(preprocess).batch(16)

# ---------------------
# 2. Model Training
# ---------------------
# Lightweight CNN suitable for edge devices
model = models.Sequential([
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=(96, 96, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(NUM_CLASSES)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train model (simplified for demo - use more epochs with real data)
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Evaluate final accuracy
test_loss, test_acc = model.evaluate(test_dataset)
print(f'\nTest accuracy: {test_acc:.2f}')

# ---------------------
# 3. TFLite Conversion
# ---------------------
# Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Quantization for size reduction
tflite_model = converter.convert()

# Save converted model
with open('recyclable_model.tflite', 'wb') as f:
    f.write(tflite_model)
    
# ---------------------
# 4. TFLite Inference
# ---------------------
# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test on sample image
sample_image, _ = preprocess(test_images[0:1], test_labels[0:1])
interpreter.set_tensor(input_details[0]['index'], sample_image)
interpreter.invoke()
tflite_output = interpreter.get_tensor(output_details[0]['index'])

# Display results
prediction = np.argmax(tflite_output[0])
confidence = tf.nn.softmax(tflite_output[0])[prediction]
print(f"\nTFLite Prediction: {CLASS_NAMES[prediction]} "
      f"(Confidence: {confidence:.2%})")

# ---------------------
# 5. Deployment Report
# ---------------------
print("\n" + "="*50)
print("Edge AI Deployment Report")
print("="*50)
print("\nModel Metrics:")
print(f"- Test Accuracy: {test_acc:.2%}")
print(f"- Model Size: {len(tflite_model)/1024:.2f} KB")

print("\nDeployment Steps:")
print("1. Train model with edge-appropriate architecture")
print("2. Quantize model using TFLiteConverter optimizations")
print("3. Transfer .tflite file to Raspberry Pi storage")
print("4. Use TFLite Interpreter API in Python/C++ application")
print("5. Process camera input through model in real-time")

print("\nEdge AI Benefits for Real-time Applications:")
print("-  Low Latency: Local processing eliminates network round-trips")
print("-  Privacy: Sensitive image data never leaves the device")
print("-  Offline Operation: Functions without internet connectivity")
print("-  Cost Efficiency: Reduces cloud processing costs")
print("-  Energy Savings: Lower power consumption than cloud transmission")

2025-07-10 12:45:31.155811: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 12:45:31.573320: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-10 12:45:31.923486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752151532.212922     188 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752151532.305646     188 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752151532.977314     188 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Epoch 1/10


2025-07-10 12:45:39.182922: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/167514a9-0b28-4d9a-b9ac-f727135ffe83/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.2496 - loss: 1.4177 - val_accuracy: 0.2450 - val_loss: 1.3867
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.2774 - loss: 1.3867 - val_accuracy: 0.2450 - val_loss: 1.3857
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.2837 - loss: 1.3831 - val_accuracy: 0.3200 - val_loss: 1.3842
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.2448 - loss: 1.3844 - val_accuracy: 0.3000 - val_loss: 1.3870
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.2674 - loss: 1.3874 - val_accuracy: 0.2850 - val_loss: 1.3846
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.2679 - loss: 1.3799 - val_accuracy: 0.3200 - val_loss: 1.3707
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.2689 - loss: 1.3841 - val_accuracy: 0.3200 - val_loss: 1.3802
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - accuracy: 0.2625 - loss: 1.3791 - val_accuracy: 0.2400 - val_loss: 1.

INFO:tensorflow:Assets written to: /tmp/tmpy3xgd01g/assets


Saved artifact at '/tmp/tmpy3xgd01g'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  128764266731952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128764266727376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128764266737936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128764266736704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128764266734768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128764267376560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128764267375152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128764267377616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128764267379024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128764267367760: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1752151572.331500     188 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1752151572.331529     188 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.



TFLite Prediction: metal (Confidence: 30.32%)

Edge AI Deployment Report

Model Metrics:
- Test Accuracy: 17.00%
- Model Size: 209.28 KB

Deployment Steps:
1. Train model with edge-appropriate architecture
2. Quantize model using TFLiteConverter optimizations
3. Transfer .tflite file to Raspberry Pi storage
4. Use TFLite Interpreter API in Python/C++ application
5. Process camera input through model in real-time

Edge AI Benefits for Real-time Applications:
- ⚡ Low Latency: Local processing eliminates network round-trips
- 🔒 Privacy: Sensitive image data never leaves the device
- 📴 Offline Operation: Functions without internet connectivity
- 💰 Cost Efficiency: Reduces cloud processing costs
- 🔋 Energy Savings: Lower power consumption than cloud transmission


2025-07-10 12:46:12.332160: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpy3xgd01g
2025-07-10 12:46:12.333243: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-07-10 12:46:12.333255: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpy3xgd01g
I0000 00:00:1752151572.338501     188 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-07-10 12:46:12.339522: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-07-10 12:46:12.374966: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpy3xgd01g
2025-07-10 12:46:12.386012: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 53857 microseconds.
2025-07-10 12:46:12.486512: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, s